In [7]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import json
from pandas.io.json import json_normalize

In [14]:
# use json_normalize to make the json readable
with open('data/rating.json') as file: 
    dog_file = json.load(file) 
dog_df = json_normalize(dog_file['dog_breeds'])
dog_df.head(10)

# clean up columns by separating breeds and traits
breeds = dog_df.columns.str.split('.').str[0]
traits = dog_df.columns.str.split('.').str[1]

#drop duplicates to establish the number of unique values for each axis
breeds = breeds.drop_duplicates()
traits = traits.drop_duplicates()

#determine how many breeds there are and how many traits are measured (214 breeds)
len(breeds), le
# new_dog_df = pd.DataFrame(index=breeds, columns=traits)
# new_dog_df

214